# 🧠 Lab 3 – Deep Learning Detection with ResNet

This notebook demonstrates image classification using a pretrained ResNet-18 model from PyTorch. The input image is sourced from Hound Rack 3 and processed through a CNN to predict its class label.


## Environment Setup

In [12]:
pip install ultralytics


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 2.4 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 2.1 MB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 468.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/38.9 MB ? eta -:--:--
    ------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import cv2
import requests
from ultralytics import YOLO
model = YOLO("yolov8n.pt")


## Load & Display Image

In [3]:
img_path = "../../images/faces/face_sample.jpg"

img_cv = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
img = Image.fromarray(img_rgb)


In [ ]:
plt.imshow(img)
plt.title("Input Image – face_sample.jpg")
plt.axis("off")
plt.show()


## Preprocess & Classify

In [5]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

input_tensor = transform(img).unsqueeze(0)


In [7]:
device = torch.device("cpu")
model = models.resnet18(pretrained=True).to(device)
model.eval()

input_tensor = input_tensor.to(device)

with torch.no_grad():
    outputs = model(input_tensor)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
    top5_prob, top5_idx = torch.topk(probabilities, 5)


C:\Users\tsion\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\tsion\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\tsion/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [01:25<00:00, 551kB/s] 


## Load Labels & Display Top-5 Predictions

In [10]:
labels_url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
labels = requests.get(labels_url).text.strip().split("\n")

for i in range(5):
    print(f"{labels[top5_idx[i]]}: {top5_prob[i].item():.4f}")


bikini: 0.2516
wig: 0.1464
maillot: 0.0608
bonnet: 0.0444
brassiere: 0.0348


In [ ]:
## ✅ Result & Reflection

The ResNet-18 model successfully classified the input image and returned the top-5 predictions with confidence scores. This demonstrates the power of CNNs in extracting hierarchical features and making accurate predictions on real-world data.

Compared to classical methods (Haar, HOG), deep learning offers:
- End-to-end learning from raw pixels
- Robustness to scale, lighting, and occlusion
- Transfer learning via pretrained models

Future enhancements may include:
- Comparing with MobileNetV2 (TensorFlow)
- Fine-tuning on a custom dataset
- Visualizing intermediate feature maps
